### Decline Curve Analisys tool

In [33]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, date
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import pylab
from pylab import rcParams

data_path = '../data/ps_owc/df.xlsx'
df_train = pd.read_excel(data_path, sheet_name='train')
df_test = pd.read_excel(data_path, sheet_name='test')

In [34]:
df_train

,Well,ST,Cum prod,Days,In prod,Q oil,wct,Top perf,Bottom perf,67db,...,64bs,75db,Plat-2,Plat-15,Plat-6,Plat-8,Mak-102,63db,61db,41db
0,67db,0,1193324,8096,1,978.804120,14.359520,-2327.00,-2362.00,1,...,456,895,1206,1476,1770,2057,2472,2330,2610,2690
1,47db,0,1385332,9039,1,276.656310,7.812500,-2332.30,-2356.30,188,...,345,780,1090,1340,1630,1910,2330,2210,2460,2560
2,68db,0,297912,2830,0,0.000000,100.000000,-2471.29,-2482.25,275,...,250,675,976,1255,1530,1820,2218,2070,2430,2960
3,64db,0,839156,7701,0,0.000000,100.000000,-2357.90,-2424.90,622,...,197,310,614,870,1160,1440,1843,1700,2000,2085
4,64bs,1,46074,943,1,724.129470,44.824001,-2335.00,-2357.00,456,...,1,448,751,1027,2625,2617,2028,1881,2170,2250
5,75db,0,913232,7366,0,0.000000,100.000000,-2367.50,-2371.50,895,...,448,1,306,580,886,1172,1574,1444,1725,1795
6,Plat-2,0,1311418,8066,1,676.000000,82.300942,-2316.00,-2331.50,1206,...,751,306,1,272,600,880,1268,1145,1440,1505
7,Plat-6,0,409876,7305,0,0.000000,100.000000,-2317.70,-2339.10,1770,...,2625,886,600,335,1,280,685,615,845,920
8,Mak-102,0,368413,8917,1,259.725098,87.058953,-2332.13,-2339.12,2472,...,2028,1574,1268,1000,685,425,1,245,220,235
9,63db,0,91609,2191,0,0.000000,100.000000,-2501.81,-2540.18,2330,...,1881,1444,1145,880,615,423,245,1,460,410


In [35]:
df_test
x_test = df_test.drop(['Well', 'wct'], axis=1)
y_test = df_test['wct']
x_test

,ST,Cum prod,Days,In prod,Q oil,Top perf,Bottom perf,67db,47db,68db,...,64bs,75db,Plat-2,Plat-15,Plat-6,Plat-8,Mak-102,63db,61db,41db
0,0,947441,7609,0,0,-2315.05,-2326.05,1476,1340,1255,...,1027,580,272,1,335,610,1000,880,1160,1220
1,0,672904,7578,1,746,-2321.98,-2386.96,2057,1910,1820,...,2617,1172,880,610,280,1,425,423,560,655


In [36]:
x = df_train.drop(['Well', 'wct'], axis=1)
y = df_train['wct']
x

,ST,Cum prod,Days,In prod,Q oil,Top perf,Bottom perf,67db,47db,68db,...,64bs,75db,Plat-2,Plat-15,Plat-6,Plat-8,Mak-102,63db,61db,41db
0,0,1193324,8096,1,978.804120,-2327.00,-2362.00,1,188,275,...,456,895,1206,1476,1770,2057,2472,2330,2610,2690
1,0,1385332,9039,1,276.656310,-2332.30,-2356.30,188,1,295,...,345,780,1090,1340,1630,1910,2330,2210,2460,2560
2,0,297912,2830,0,0.000000,-2471.29,-2482.25,275,295,1,...,250,675,976,1255,1530,1820,2218,2070,2430,2960
3,0,839156,7701,0,0.000000,-2357.90,-2424.90,622,480,456,...,197,310,614,870,1160,1440,1843,1700,2000,2085
4,1,46074,943,1,724.129470,-2335.00,-2357.00,456,345,250,...,1,448,751,1027,2625,2617,2028,1881,2170,2250
5,0,913232,7366,0,0.000000,-2367.50,-2371.50,895,780,675,...,448,1,306,580,886,1172,1574,1444,1725,1795
6,0,1311418,8066,1,676.000000,-2316.00,-2331.50,1206,1090,976,...,751,306,1,272,600,880,1268,1145,1440,1505
7,0,409876,7305,0,0.000000,-2317.70,-2339.10,1770,1630,1530,...,2625,886,600,335,1,280,685,615,845,920
8,0,368413,8917,1,259.725098,-2332.13,-2339.12,2472,2330,2218,...,2028,1574,1268,1000,685,425,1,245,220,235
9,0,91609,2191,0,0.000000,-2501.81,-2540.18,2330,2210,2070,...,1881,1444,1145,880,615,423,245,1,460,410


In [37]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x)

x_test = scaler.transform(df_test.drop(['Well', 'wct'], axis=1))

model = RandomForestRegressor(random_state=42, max_depth=14)
model.fit(x_train, y)

y_pred = model.predict(x_test)

y_pred_train = model.predict(x_train)

r2_train = r2(y, y_pred_train)
print(f'R2 train: {r2_train.round(4)}')
model

R2 train: 0.8714


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=14, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [38]:
df_y_test = pd.DataFrame({'Well': df_test['Well'], 'wct': y_pred})
df_y_test

,Well,wct
0,Plat-15,96.732339
1,Plat-8,85.819991


In [39]:
model.feature_importances_
feature_importances = pd.DataFrame()
feature_importances['importance'] = model.feature_importances_
feature_importances['feature_name'] = x.columns.tolist()
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
feature_importances

,importance,feature_name
15,0.216304,Plat-6
16,0.153696,Plat-8
4,0.129701,Q oil
7,0.107818,67db
2,0.088313,Days
8,0.070078,47db
19,0.040483,61db
9,0.031322,68db
18,0.030909,63db
0,0.029220,ST


In [40]:
def evaluate_preds(true_values, pred_values):
    print("R2:\t" + str(round(r2(true_values, pred_values), 3)) + "\n" +
          "MAE:\t" + str(round(mae(true_values, pred_values), 0)) + "\n" +
          "MSE:\t" + str(round(mse(true_values, pred_values), 0))) 
    plt.figure(figsize=(6,6))
    sns.scatterplot(x=pred_values, y=true_values, s=1)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('True vs Predicted values')
    plt.show()

evaluate_preds(y_valid.values.flatten(), y_pred.flatten())
evaluate_preds(y_train.values.flatten(), y_pred_train.flatten())

NameError: name 'y_valid' is not defined